In [1]:
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from geojson_pydantic import FeatureCollection
from typing import Any, Sequence, Type
import requests
from langchain.tools.requests.tool import RequestsGetTool

BASE_URL = 'http://localhost:9000'

def describe_collection(url: str, relevant_properties: Sequence[str]) -> str:
    res = requests.get(url).json()
    return (
        f"Collection ID: {res['id']}\n"
        f"Description: {res['description']}\n"
        f"Geometry Type: {res['geometrytype']}\n"
        f"Relevant Properties: {[prop for prop in res['properties'] if prop['name'] in relevant_properties or len(prop) == 0]}\n"
    )

print(describe_collection(
    'http://localhost:9000/collections/public.5001elveg2.0l.json', ['koordh', 'fartsgrens']))

class RelevantCollectionProperties(BaseModel):
    collection_id: str = Field(..., description='ID of the collection')
    relevant_properties: Sequence[str] = Field(..., description="Properties in the collection that can be used to answer the user's query")

class RelevantCollectionsAndPropertiesInput(BaseModel):
    collections_with_props: Sequence[RelevantCollectionProperties]

class RelevantCollectionsAndPropertiesTool(BaseTool):
    name: str = 'relevant_collections_and_properties'
    args_schema: Type[BaseModel] = RelevantCollectionsAndPropertiesInput
    description: str = 'Use this to get a concise textual description of relevant collections and properties within these.'

    def _run(self, collections_with_props: Sequence[RelevantCollectionProperties], *args: Any, **kwargs: Any) -> Any:
        return [describe_collection(f'{BASE_URL}/collections/{cwp.collection_id}', cwp.relevant_properties) for cwp in collections_with_props]
    



Collection ID: public.5001elveg2.0l
Description: Elveg 2.0 - Vegnettsdatasett som omfatter alle kjørbare veger som er lengre enn 50 meter, eller del av et nettverk, samt gang- og sykkelveger og sykkelveger representert som veglenkegeometri.
Geometry Type: MultiLineString
Relevant Properties: [{'name': 'koordh', 'type': 'number', 'description': ''}, {'name': 'fartsgrens', 'type': 'number', 'description': ''}]



In [6]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.tools.requests.tool import RequestsGetTool
from langchain.requests import RequestsWrapper

load_dotenv('../../../.env')

system_message = """Create a computational graph for a GIS-related question.
All operation nodes should be followed by an intermediary node.
"""

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

# I should then go to http://localhost:9000/collections/{collection_id} to see which properties are relevant. 
AI_SUFFIX = """I should look through ALL collections available at http://localhost:9000/collections to see which are usable for the problem at hand.
I should then get a concise description of these collections and the relevant properties of these. 

Here is the query from the user: 
{user_query}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_message),
        AIMessage(content=AI_SUFFIX),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt.partial(
    user_query='Get me the road segments of the noisiest roads in Trondheim.')

tools = [RelevantCollectionsAndPropertiesTool(), RequestsGetTool(
    requests_wrapper=RequestsWrapper())]

agent = create_openai_tools_agent(llm=llm, prompt=prompt, tools=tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

res = agent_executor.invoke()

# res = agent_executor.invoke(
#     {'user_query': 'Get me the road segments of the noisiest roads in Trondheim.'})

TypeError: Chain.invoke() missing 1 required positional argument: 'input'